# 1. Grid Search

## 使用单线程对文本分类的朴素贝叶斯模型的超参数模型执行组合网格搜索

In [1]:
# 从sklearn.datasets中导入20类新闻文本抓取器
from sklearn.datasets import fetch_20newsgroups
import numpy as np

# 使用新闻抓取器从互联网上下载所有数据
news = fetch_20newsgroups(subset='all')

In [2]:
# 从sklearn.cross_validation导入train_test_split用来分割数据
from sklearn.cross_validation import train_test_split

# 对前3000条新闻文本进行数据分割，25%的文本用于测试
X_train, X_test, y_train, y_test = train_test_split(news.data[:3000], news.target[:3000], test_size=0.25, random_state=33)

D:\Program Soft\Anaconda\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
# 导入支持向量机(分类)模型
from sklearn.svm import SVC
# 导入TfidfVectorizer文本抽取器
from sklearn.feature_extraction.text import TfidfVectorizer
# 导入Pipeline
from sklearn.pipeline import Pipeline

# 使用Pipeline简化系统搭建流程，将文本抽取与分类器模型串联起来
clf = Pipeline([('vect', TfidfVectorizer(stop_words='english', analyzer='word')), ('svc', SVC())])

# 需要试验2个的超参数的个数分别是4、3，svc_gamma的参数共有10^-2,10^-1,...
# 一共有12种超参数组合，12个不同参数下的模型
parameters = {'svc__gamma': np.logspace(-2,1,4), 'svc__C': np.logspace(-1,1,3)}


# 从sklearn.grid_search中导入网格搜索模块GridSearchCV
from sklearn.grid_search import GridSearchCV

# 将12组参数组合以及初始化的Pipeline包括3折交叉验证的要求全部告知GridSearchCV
# 注意设定refit=True
gs = GridSearchCV(clf, parameters, verbose=2, refit=True, cv=3)

# 执行单线程网格搜索
%time _ = gs.fit(X_train, y_train)
gs.best_params_, gs.best_score_

# 输出最佳模型在测试集上的准确性
print(gs.score(X_test, y_test))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] svc__C=0.1, svc__gamma=0.01 .....................................
[CV] ............................ svc__C=0.1, svc__gamma=0.01 -   8.4s
[CV] svc__C=0.1, svc__gamma=0.01 .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.5s remaining:    0.0s


[CV] ............................ svc__C=0.1, svc__gamma=0.01 -   8.1s
[CV] svc__C=0.1, svc__gamma=0.01 .....................................
[CV] ............................ svc__C=0.1, svc__gamma=0.01 -   8.9s
[CV] svc__C=0.1, svc__gamma=0.1 ......................................
[CV] ............................. svc__C=0.1, svc__gamma=0.1 -   7.9s
[CV] svc__C=0.1, svc__gamma=0.1 ......................................
[CV] ............................. svc__C=0.1, svc__gamma=0.1 -  11.3s
[CV] svc__C=0.1, svc__gamma=0.1 ......................................
[CV] ............................. svc__C=0.1, svc__gamma=0.1 -   9.2s
[CV] svc__C=0.1, svc__gamma=1.0 ......................................
[CV] ............................. svc__C=0.1, svc__gamma=1.0 -   9.4s
[CV] svc__C=0.1, svc__gamma=1.0 ......................................
[CV] ............................. svc__C=0.1, svc__gamma=1.0 -   8.1s
[CV] svc__C=0.1, svc__gamma=1.0 ......................................
[CV] .

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  5.1min finished


Wall time: 5min 21s
0.822666666667


## 2. 并行搜索

In [14]:
# 将12组参数组合以及初始化的Pipeline包括3折交叉验证的要求全部告知GridSearchCV
# 注意设定refit=True
gs = GridSearchCV(clf, parameters, verbose=2, refit=True, cv=3, n_jobs=-1)

# 执行单线程网格搜索
%time _ = gs.fit(X_train, y_train)
gs.best_params_, gs.best_score_

# 输出最佳模型在测试集上的准确性
print(gs.score(X_test, y_test))

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  2.2min finished


Wall time: 2min 27s
0.822666666667
